In [1]:
from langchain.docstore.document import Document

In [11]:
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

In [12]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [2]:
import pandas as pd
data = pd.read_csv('data/winemag-data_first150k.csv')
data = data.head(100)
data = data.drop(data.columns[0], axis=1)


In [3]:
docs = []
for i in range(len(data)):
    meta_data = {
        'country':data['country'][i],
        'variety':data['variety'][i]
    }
    page_content = data['description'][i]
    doc = Document(
        metadata=meta_data,
        page_content=page_content
    )
    docs.append(doc)

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

In [5]:
embeddings = HuggingFaceEmbeddings()

/Users/jon/miniconda3/envs/langchain/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain.vectorstores import Chroma

In [15]:
!rm -rf temp
!mkdir temp
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory='temp'
)

In [31]:
docs[0]

Document(page_content='This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022–2030.', metadata={'country': 'US', 'variety': 'Cabernet Sauvignon'})

In [7]:
from langchain.chains.query_constructor.base import AttributeInfo
meta_data_description = [
    AttributeInfo(
        name='country',
        description='country or region the wine is from',
        type='string'
    ),
    AttributeInfo(
        name='variety',
        description='type or grape or grape blend of wine',
        type='string'
    )
]

In [8]:
from langchain.retrievers.self_query.base import SelfQueryRetriever

In [44]:
from langchain.llms import HuggingFaceHub
document_content_description = "wine reviews"
llm = HuggingFaceHub(repo_id='google/flan-t5-xxl')

In [45]:
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    meta_data_description,
    verbose=True
)

In [46]:
question = "what is said about Merlot"

In [47]:
retriever.get_relevant_documents(question)

OutputParserException: Parsing text
json  "query": "Merlot", "filter":
 raised following error:
Got invalid JSON object. Error: Expecting value: line 1 column 1 (char 0)